In [133]:
import requests
import pandas as pd
import numpy as np

In [183]:
def data_make(link):
    # Function to make dataframe from InfoSparks link
    csv = requests.get(link)
    csv_split = csv.text.replace("$","").replace("\r","").split("\n")
    csv_split = [x for x in csv_split if x != '']
    metric = [x for x in csv_split if 'Metric:' in x][0].split(',')[1:][0]
    segments = [x for x in csv_split if 'Segments:' in x][0].split(',')[1:]
    time_calculation = [x for x in csv_split if 'Time Calculation:' in x][0].split(',')[1:][0]
    filters = {segment:[x for x in csv_split if segment+':' in x][0].split(', ')[1:] for segment in segments}
    data_locator = "Date,"
    for segment in segments:
        data_locator += "\"" + segment + "\"" + ","
    data_index = csv_split.index(data_locator)
    data = csv_split[data_index + 1 : -1]
    data = [string.split(",")[:-1] for string in data]
    data = np.array(data).transpose()
    df = pd.DataFrame(columns=["Date", metric, "Location", "Time Calculation"])
    for i, segment in enumerate(segments):
        data_dict = {'Date':data[0], metric:[float(x) if x != '' else np.nan for x in data[i+1] ], "Location":segment, "Time Calculation": time_calculation}
        for filter in filters[segment]:
            data_dict["#"+filter] = "Yes"
        df = pd.concat([df, pd.DataFrame(data_dict)])

    df['Date'] = pd.to_datetime(df['Date'])
    df['Date'] = df['Date'].apply(lambda x: x.strftime('%Y-%m'))

    return df


In [214]:
def merge_two_dfs(df1, df2):
    # merge two dfs into one big dfs, combining on filters, date, location and time calculation
    add_filter_cols = [col for col in df2.columns if col not in df1.columns and "#" in col]
    for filter in add_filter_cols:
        df1[filter] = "No"

    return df1.merge(df2, how='outer')

def merge_dfs(dfs):
    # merge many dfs into one
    if len(dfs) > 1:
        df = dfs[0]
        for dff in dfs[1:]:
            df = merge_two_dfs(df, dff)
        return df
    else:
        return dfs[0]
    
def links_to_data(links):
    if type(links[0]) == list:
        dfs = [data_make(link) for link in links[0]]
        df = merge_dfs(dfs)
        for link_list in links[1:]:
            dfs = [data_make(link) for link in link_list]
            df = df.merge(merge_dfs(dfs), how='outer')
        return df
    else:
        dfs = [data_make(link) for link in links]
        return merge_dfs(dfs)

In [215]:

link1 = [
    # These links are all the most granular live data on InfoSpark about Edmonds, Shoreline, Mukilteo, and Lynnwood. No filters, all features, Monthly calculations
    "https://nwmls.stats.showingtime.com/infoserv/s-v1/JUr9-RbE",
    "https://nwmls.stats.showingtime.com/infoserv/s-v1/JUrh-Xcz",
    "https://nwmls.stats.showingtime.com/infoserv/s-v1/JUr2-i1W",
    "https://nwmls.stats.showingtime.com/infoserv/s-v1/JUrT-JCM",
    "https://nwmls.stats.showingtime.com/infoserv/s-v1/JUrb-J5Z",
    "https://nwmls.stats.showingtime.com/infoserv/s-v1/JUr0-pAP",
    "https://nwmls.stats.showingtime.com/infoserv/s-v1/JUrf-yYx",
    "https://nwmls.stats.showingtime.com/infoserv/s-v1/JUrP-1f7",
    "https://nwmls.stats.showingtime.com/infoserv/s-v1/JUrn-2jA",
    "https://nwmls.stats.showingtime.com/infoserv/s-v1/JUrV-Vwt",
    "https://nwmls.stats.showingtime.com/infoserv/s-v1/JUrc-v93",
    "https://nwmls.stats.showingtime.com/infoserv/s-v1/JUr8-4iG",
    "https://nwmls.stats.showingtime.com/infoserv/s-v1/JUDB-9A1",
    "https://nwmls.stats.showingtime.com/infoserv/s-v1/JUru-xDc",
]
link2 = [
    # next is the same for Lake Forest Park, Mountlake Terrace, Kenmore, and Bothell
    "https://nwmls.stats.showingtime.com/infoserv/s-v1/JUDl-Oll",
    "https://nwmls.stats.showingtime.com/infoserv/s-v1/JUDZ-7d8",
    "https://nwmls.stats.showingtime.com/infoserv/s-v1/JUDj-9GJ",
    "https://nwmls.stats.showingtime.com/infoserv/s-v1/JUDf-gBX",
    "https://nwmls.stats.showingtime.com/infoserv/s-v1/JUDP-i9m",
    "https://nwmls.stats.showingtime.com/infoserv/s-v1/JUDY-axI",
    "https://nwmls.stats.showingtime.com/infoserv/s-v1/JUDH-TzJ",
    "https://nwmls.stats.showingtime.com/infoserv/s-v1/JUDF-inz",
    "https://nwmls.stats.showingtime.com/infoserv/s-v1/JUDE-vQj",
    "https://nwmls.stats.showingtime.com/infoserv/s-v1/JUDR-m03",
    "https://nwmls.stats.showingtime.com/infoserv/s-v1/JUD8-lQN",
    "https://nwmls.stats.showingtime.com/infoserv/s-v1/JUDw-Pkn",
    "https://nwmls.stats.showingtime.com/infoserv/s-v1/JUDS-ijU",
    "https://nwmls.stats.showingtime.com/infoserv/s-v1/JUDO-8Kf",
]
df = links_to_data([link1, link2])


In [224]:
df.describe()

,Average Sales Price,New Listings,Homes for Sale,Pending Sales,Closed Sales,Average Days on Market,Months Supply of Inventory (Pending),Months Supply of Inventory (Closed),Average Percent of Last List Price,Average Percent of Last Original Price,Average Price Per Square Foot,Dollar Volume of Closed Sales,Average Showings to Pending,Showings Per Listing
count,1.664000e+03,1664.000000,1664.000000,1664.000000,1664.000000,1664.000000,1663.000000,1663.000000,1664.000000,1664.000000,1664.000000,1.664000e+03,416.000000,416.000000
mean,5.020613e+05,90.102764,192.661659,65.019832,65.037260,43.596154,3.473542,3.481539,1.002757,0.982463,259.163462,3.308391e+07,17.485577,11.046154
std,2.014210e+05,76.786549,216.459988,56.365455,56.637912,28.020809,3.120583,3.143303,0.034706,0.048799,93.491555,3.448561e+07,12.847569,9.751129
min,1.446290e+05,4.000000,0.000000,2.000000,2.000000,4.000000,0.000000,0.000000,0.885000,0.801000,102.000000,7.770000e+05,1.000000,0.300000
25%,3.598765e+05,32.000000,52.000000,24.000000,24.000000,21.000000,1.200000,1.200000,0.982750,0.951000,189.750000,1.090746e+07,4.000000,1.500000
50%,4.447365e+05,60.000000,111.000000,43.000000,42.000000,37.000000,2.300000,2.300000,0.996000,0.982000,232.000000,2.089048e+07,18.000000,9.400000
75%,6.208810e+05,128.000000,241.000000,91.000000,91.000000,61.000000,5.050000,5.100000,1.013000,1.005000,314.000000,4.299501e+07,25.000000,15.725000
max,1.352763e+06,453.000000,1222.000000,289.000000,307.000000,175.000000,16.200000,17.700000,1.264000,1.264000,576.000000,2.476258e+08,88.000000,55.100000


# Cut the data!

In [263]:
dff = df.loc[df.Date >= "2023-01"].groupby(['Location']).sum()
dff["Average Sales Price"] = dff["Dollar Volume of Closed Sales"] / dff["Closed Sales"]
dff["Average Sales Price Color"] = dff["Average Sales Price"] - dff["Average Sales Price"].mean() / 1.5
dff["Location"] = dff.index
dff

,Average Sales Price,New Listings,Homes for Sale,Pending Sales,Closed Sales,Average Days on Market,Months Supply of Inventory (Pending),Months Supply of Inventory (Closed),Average Percent of Last List Price,Average Percent of Last Original Price,Average Price Per Square Foot,Dollar Volume of Closed Sales,Average Showings to Pending,Showings Per Listing,Average Sales Price Color,Location
Location,,,,,,,,,,,,,,,,
Bothell,9.475092e+05,532.0,474.0,523.0,476.0,137.0,3.0,3.0,3.972,3.885,1721.0,451014386.0,95.0,47.0,388915.199298,Bothell
Edmonds,9.082055e+05,224.0,220.0,225.0,181.0,138.0,3.5,3.3,3.976,3.898,1821.0,164385188.0,85.0,42.4,349611.443576,Edmonds
Kenmore,1.004691e+06,162.0,178.0,137.0,107.0,125.0,6.0,5.8,4.003,3.921,1771.0,107501945.0,86.0,38.4,446097.059778,Kenmore
Lake Forest Park,9.567050e+05,36.0,31.0,32.0,32.0,117.0,2.4,2.3,4.105,4.012,1702.0,30614561.0,132.0,83.2,398111.016262,Lake Forest Park
Lynnwood,7.023898e+05,353.0,329.0,367.0,323.0,137.0,3.1,3.1,3.977,3.900,1508.0,226871919.0,80.0,40.9,143795.827117,Lynnwood
Mountlake Terrace,6.275696e+05,81.0,61.0,74.0,59.0,114.0,2.8,2.7,3.956,3.906,1755.0,37026609.0,88.0,46.6,68975.629080,Mountlake Terrace
Mukilteo,7.293451e+05,89.0,92.0,93.0,77.0,140.0,4.0,3.9,3.987,3.931,1728.0,56159575.0,69.0,35.3,170751.114882,Mukilteo
Shoreline,8.267128e+05,267.0,368.0,242.0,186.0,139.0,6.8,6.4,4.019,3.921,1862.0,153768578.0,86.0,35.7,268118.769958,Shoreline


# Plot the data!

In [292]:
import plotly.express as px
import plotly.io as pio
# plot theme
pio.templates.default = 'seaborn'

fig = px.scatter(dff, x='Average Sales Price', y="Closed Sales", range_x=[500000, 1100000], color="Location", text="Location", size="Average Sales Price Color", size_max=35, width = 700, height=400 )
fig.update_layout(title_text="Average Sales Price and Closed Sales in 2023", title_x=0.5, showlegend=False)
fig.update_layout(
    font_family="Merriweather"
)

def improve_text_position(location):
    if location in ["Lake Forest Park","Bothell"]:
        return 'middle left'
    else:
        return 'top center'

for location in dff['Location']:
    fig.update_traces(selector = dict(name = location), textposition=improve_text_position(location))

fig.show()


In [289]:
dff[['Average Sales Price', 'Closed Sales']].to_csv()

'Location,Average Sales Price,Closed Sales\r\nBothell,947509.2142857143,476.0\r\nEdmonds,908205.4585635359,181.0\r\nKenmore,1004691.0747663551,107.0\r\nLake Forest Park,956705.03125,32.0\r\nLynnwood,702389.8421052631,323.0\r\nMountlake Terrace,627569.6440677966,59.0\r\nMukilteo,729345.1298701299,77.0\r\nShoreline,826712.7849462365,186.0\r\n'